In [ ]:
import arcpy
import os

# Define paths and parameters
project_path = r"C:\Projects\Vancouver Parks\Vancouver Parks GIS\Vancouver Parks GIS.aprx"
output_base_folder = r"C:\Projects\Vancouver Parks\Figures""C:\\Projects\\Vancouver Parks\\Vancouver Parks GIS\\Flood Extent Delineation.ipynb"
layout_name = "Flood Extent MapSeries"  # Your layout name

# List of flood extent layers (matches the names in the screenshot)
flood_layers = [
    "Flood Extent 100 Year RP",
    "Flood Extent 100 Year RP + 0.5m SLR",
    "Flood Extent 100 Year RP + 1m SLR",
    "Flood Extent 100 Year RP + 2m SLR"
]

# Colors for individual exports (one layer at a time)
individual_fill_color = {"RGB": [0, 77, 168, 179]}  # #004DA8
individual_boundary_color = {"RGB": [0, 38, 115, 179]}  # #002673

# Colors for combined export (all layers on)
combined_colors = {
    "Flood Extent 100 Year RP": {
        "fill": {"RGB": [0, 77, 168, 179]},     # #004DA8
        "boundary": {"RGB": [0, 38, 115, 179]}  # #002673
    },
    "Flood Extent 100 Year RP + 0.5m SLR": {
        "fill": {"RGB": [56, 168, 0, 179]},     # green
        "boundary": {"RGB": [38, 115, 0, 179]}  # darker green)
    },
    "Flood Extent 100 Year RP + 1m SLR": {
        "fill": {"RGB": [255, 255, 0, 179]},     # (professional yellow)
        "boundary": {"RGB": [230, 230, 0, 179]}  # (darker yellow)
    },
    "Flood Extent 100 Year RP + 2m SLR": {
        "fill": {"RGB": [230, 150, 30, 179]},    # (orange)
        "boundary": {"RGB": [168, 112, 0, 179]}  # (darker orange)
    }
}

# Open the ArcGIS Pro project
p = arcpy.mp.ArcGISProject(project_path)

# Debug: Print all layout names to verify the correct name
print("Available layouts:", [layout.name for layout in p.listLayouts()])

# Get the layout
layout = p.listLayouts(layout_name)[0]

# Debug: Print all map frames in the layout
print("Available map frames in layout:", [elem.name for elem in layout.listElements("MAPFRAME_ELEMENT")])

# Get the map frame and map
map_frame = None
for elem in layout.listElements("MAPFRAME_ELEMENT"):
    if elem.name == "Master Frame":  # Check for the specific map frame name
        map_frame = elem
        break
if map_frame is None:
    raise Exception("Map frame 'Master Frame' not found in the layout.")
map = map_frame.map

# Debug: Print the map name
#print("Map linked to map frame:", map.name if map else "No map linked")

# Debug: Print all top-level layers in the map
layers = map.listLayers()
layer_names = []
for lyr in layers:
    try:
        layer_names.append(lyr.name)
    except AttributeError:
        layer_names.append(f"<Layer with no name attribute: {type(lyr).__name__}>")
#print("Top-level layers in map:", layer_names)

# Get the Flood Extent group layer
group_layer = None
for lyr in layers:
    try:
        if lyr.name == "Flood Extent":
            group_layer = lyr
            break
    except AttributeError:
        continue
if group_layer is None:
    raise Exception("Group layer 'Flood Extent' not found in the map.")

# Debug: Print all layers in the group to verify names
group_layer_names = []
for lyr in group_layer.listLayers():
    try:
        group_layer_names.append(lyr.name)
    except AttributeError:
        group_layer_names.append(f"<Layer with no name attribute: {type(lyr).__name__}>")
print("Layers in 'Flood Extent' group:", group_layer_names)

# Part 1: Export each flood extent layer individually
for flood_layer_name in flood_layers:
    # Create a dedicated folder for this flood extent
    output_folder = os.path.join(output_base_folder, flood_layer_name)
    os.makedirs(output_folder, exist_ok=True)

    # Turn off all flood layers within the group
    for lyr in group_layer.listLayers():
        try:
            if lyr.name in flood_layers:
                lyr.visible = False
        except AttributeError:
            continue

    # Turn on the current flood layer and set symbology
    current_layer = None
    for lyr in group_layer.listLayers():
        try:
            if lyr.name == flood_layer_name:
                current_layer = lyr
                break
        except AttributeError:
            continue
    if current_layer is None:
        raise Exception(f"Flood layer '{flood_layer_name}' not found in the 'Flood Extent' group.")
    current_layer.visible = True

    # Apply symbology for individual export
    sym = current_layer.symbology
    if hasattr(sym, "renderer"):
        renderer = sym.renderer
        if renderer.type == "SimpleRenderer":
            symbol = renderer.symbol
            symbol.color = individual_fill_color
            symbol.outlineColor = individual_boundary_color
            symbol.outlineWidth = 0.7  # Adjust as needed
            sym.renderer = renderer
            current_layer.symbology = sym

    # Access the map series
    ms = layout.mapSeries
    if ms.enabled:
        # Export to a single PDF
        pdf_output = os.path.join(output_folder, f"{flood_layer_name} MapSeries.pdf")
        ms.exportToPDF(
            pdf_output,
            page_range_type="ALL",
            resolution=300,  # Lower resolution for testing
            image_compression="JPEG",  # Avoid compression artifacts
            output_as_image=True,  # Preserve vector graphics
            convert_markers=False,  # Prevent marker conversion
            embed_fonts=True  # Ensure fonts are embedded
        )

        # Export each page to individual PNGs
        for page_num in range(1, ms.pageCount + 1):
            ms.currentPageNumber = page_num
            page_name = ms.pageRow.ParkCoverage  # Use the ParkCoverage field directly
            # Replace any invalid characters in the page name for file naming
            page_name = page_name.replace("/", "_").replace("\\", "_")
            png_output = os.path.join(output_folder, f"{flood_layer_name} - {page_name}.png")
            layout.exportToPNG(png_output, resolution=300)

# Part 2: Export all flood extent layers together
combined_output_folder = os.path.join(output_base_folder, "All Flood Extents")
os.makedirs(combined_output_folder, exist_ok=True)

# Turn on all flood layers and set symbology
for flood_layer_name in flood_layers:
    layer = None
    for lyr in group_layer.listLayers():
        try:
            if lyr.name == flood_layer_name:
                layer = lyr
                break
        except AttributeError:
            continue
    if layer is None:
        raise Exception(f"Flood layer '{flood_layer_name}' not found in the 'Flood Extent' group.")
    layer.visible = True

    # Apply symbology for combined export
    sym = layer.symbology
    if hasattr(sym, "renderer"):
        renderer = sym.renderer
        if renderer.type == "SimpleRenderer":
            symbol = renderer.symbol
            symbol.color = combined_colors[flood_layer_name]["fill"]
            symbol.outlineColor = combined_colors[flood_layer_name]["boundary"]
            symbol.outlineWidth = 0.7  # Adjust as needed
            sym.renderer = renderer
            layer.symbology = sym

# Export the combined map series
if ms.enabled:
    # Export to a single PDF
    pdf_output = os.path.join(combined_output_folder, "All Flood Extents MapSeries.pdf")
    ms.exportToPDF(
        pdf_output,
        page_range_type="ALL",
        resolution=300,  # Lower resolution for testing
        image_compression="JPEG",  # Avoid compression artifacts
        output_as_image=True,  # Preserve vector graphics
        convert_markers=False,  # Prevent marker conversion
        embed_fonts=True  # Ensure fonts are embedded
    )

    # Export each page to individual PNGs
    for page_num in range(1, ms.pageCount + 1):
        ms.currentPageNumber = page_num
        page_name = ms.pageRow.ParkCoverage  # Use the ParkCoverage field directly
        # Replace any invalid characters in the page name for file naming
        page_name = page_name.replace("/", "_").replace("\\", "_")
        png_output = os.path.join(combined_output_folder, f"All Flood Extents - {page_name}.png")
        layout.exportToPNG(png_output, resolution=300)

print("Export process completed!")

Available layouts: ['Geology MapSeries', 'Infrastructure MapSeries', 'DTM MapSeries', 'Overview MapSeries', 'Habitat MapSeries', 'Flood Extent MapSeries', 'NoFrame Topographic Tabloid', 'VFPA Concept', 'NoFrame Satellite Tabloid']
Available map frames in layout: ['Overview Frame', 'Master Frame']
Layers in 'Flood Extent' group: ['Flood Extent 100 Year RP', 'Flood Extent 100 Year RP + 0.5m SLR', 'Flood Extent 100 Year RP + 1m SLR', 'Flood Extent 100 Year RP + 2m SLR']
